Installation Requirements pour la 1er fois

In [10]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Import nécessaire

In [11]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

Connexion + sécurité pour éviter d'autre crash

In [12]:
# Connexion
con = duckdb.connect("football.duckdb")

# Sécurité mémoire
con.execute("PRAGMA threads=4;")
con.execute("PRAGMA memory_limit='4GB';")

print("DuckDB ready")

DuckDB ready


In [13]:
audit = con.execute("""
    SELECT 
        COUNT(*) AS n_rows,
        COUNT(DISTINCT player_id) AS n_sensors,
        MIN(time_utc) AS start,
        MAX(time_utc) AS end
    FROM tracking
""").df()

print(audit)

      n_rows  n_sensors               start                 end
0  166577874         29 2019-07-04 08:52:15 2021-01-15 09:58:22


Création nouvelle table avec information nécessaire et modification de nom plus logique 
=> player_id devient sensor_id
=> Suppression Info liée au Heart rate

In [14]:
con.execute("""
CREATE OR REPLACE VIEW tracking_v1 AS
SELECT
    session_type,
    session_date,
    player_id AS sensor_id,
    time_utc,
    x_pos,
    y_pos,
    latitude,
    longitude,
    speed_kmh
FROM tracking
""")

Ajout Colonne player_id, qui seront intégrés par la suite via le fichier csv summarize

In [15]:
con.execute("""
CREATE OR REPLACE VIEW tracking_v2 AS
SELECT
    *,
    NULL::VARCHAR AS player_id
FROM tracking_v1
""")

Réorganisation fichier csv pour ne garder que les sensors la date et le nom des joueurs

In [16]:
df_map = (
    pd.read_csv("data/summary.csv", sep=",")
    .assign(
        session_date=lambda x: pd.to_datetime(
            x["date"],
            format="%Y-%m-%d %H:%M:%S",
            errors="raise"   
        ).dt.date
    )
    .rename(columns={
        "Sensor": "sensor_id",
        "last_name": "player_id",
        "feature_dec_3.5mss": "feature_dec_3_5mss",
        "feature_acc_3.5mss": "feature_acc_3_5mss"
    })
    [["session_date", "sensor_id", "player_id", "position",'feature_distance', 'feature_d_speed_0_5',
       'feature_d_speed_5_10', 'feature_d_speed_10_15',
       'feature_d_speed_15_21', 'feature_d_speed_21_24',
       'feature_d_speed_24_30', 'feature_d_speed_over30', 'feature_hid_d_15',
       'feature_hid_d_21', 'feature_acc_3mss', 'feature_dec_3mss',
       'feature_acc_3_5mss', 'feature_dec_3_5mss', 'feature_acc_4mss',
       'feature_dec_4mss', 'feature_sprint_over24', 'feature_avg_speed',
       'feature_max_speed', 'feature_max_acc']]
    .drop_duplicates()
)

print(df_map.head())

  session_date  sensor_id  player_id        position  feature_distance  \
0   2020-02-01          4   KNEZEVIC     center back             10.88   
1   2020-02-01          5      ALVES     center back             10.52   
2   2020-02-01          9  MARGIOTTA      ran center              7.46   
3   2020-02-01         10      ELEKE  center forward              3.75   
4   2020-02-01         11    SCHURPF    left forward             11.38   

   feature_d_speed_0_5  feature_d_speed_5_10  feature_d_speed_10_15  \
0                 2.33                  3.91                   3.18   
1                 2.39                  3.76                   2.89   
2                 1.50                  2.30                   1.97   
3                 0.85                  1.33                   0.91   
4                 2.16                  3.75                   3.23   

   feature_d_speed_15_21  feature_d_speed_21_24  ...  feature_acc_3mss  \
0                   1.22                   0.16  ...  

In [17]:
con.register("sensor_player_map", df_map)

con.execute("""
CREATE OR REPLACE VIEW mapping_v AS
SELECT * FROM sensor_player_map
""")

In [18]:
con.execute("""
CREATE OR REPLACE VIEW tracking_v3 AS
SELECT
    t.*,
    m.player_id AS player_name,
    m.position,
    m.feature_distance,
    m.feature_d_speed_0_5,
    m.feature_d_speed_5_10,
    m.feature_d_speed_10_15,
    m.feature_d_speed_15_21,
    m.feature_d_speed_21_24,
    m.feature_d_speed_24_30,
    m.feature_d_speed_over30,
    m.feature_hid_d_15,
    m.feature_hid_d_21,
    m.feature_acc_3mss,
    m.feature_dec_3mss,
    m.feature_acc_3_5mss,
    m.feature_dec_3_5mss,
    m.feature_acc_4mss,
    m.feature_dec_4mss,
    m.feature_sprint_over24,
    m.feature_avg_speed,
    m.feature_max_speed,
    m.feature_max_acc
FROM tracking_v2 t
LEFT JOIN mapping_v m
ON t.sensor_id = m.sensor_id
AND t.session_date = m.session_date
""")



In [19]:
unmapped = con.execute("""
SELECT
    session_date,
    sensor_id,
    COUNT(*) AS n_rows
FROM tracking_v3
WHERE player_name IS NULL
GROUP BY session_date, sensor_id
ORDER BY n_rows DESC
""").df()

print(unmapped.head())

  session_date  sensor_id  n_rows
0   2019-10-10          7   36851


In [20]:
con.execute("""
CREATE OR REPLACE TABLE tracking_clean AS
SELECT *
FROM tracking_v3
WHERE player_name IS NOT NULL;
""")

OutOfMemoryException: Out of Memory Error: Allocation failure

In [ ]:
summary = con.execute("""
SELECT
    COUNT(*) AS n_rows,
    COUNT(DISTINCT player_name) AS n_players,
    COUNT(DISTINCT position) AS n_positions,
    COUNT(DISTINCT session_date) AS n_sessions,
    COUNT(DISTINCT sensor_id) AS n_sensors
FROM tracking_clean
""").df()

print(summary)

      n_rows  n_players  n_positions  n_sessions  n_sensors
0  172879917         42           16         258         29


In [ ]:
head_clean = con.execute("""
SELECT *
FROM tracking_clean
LIMIT 5
""").df()
print(head_clean)

  session_type session_date  sensor_id                time_utc      x_pos  \
0         game   2020-01-11         17 2020-01-11 15:26:35.000  74.961349   
1         game   2020-01-11         17 2020-01-11 15:26:35.100  74.954948   
2         game   2020-01-11         17 2020-01-11 15:26:35.200  74.966191   
3         game   2020-01-11         17 2020-01-11 15:26:35.300  74.962717   
4         game   2020-01-11         17 2020-01-11 15:26:35.400  75.025792   

       y_pos   latitude  longitude  speed_kmh player_id  ... feature_acc_3mss  \
0  54.050424  36.500174  -4.948796    1.20672      None  ...             38.0   
1  54.402196  36.500175  -4.948793    2.34792      None  ...             38.0   
2  54.685193  36.500175  -4.948790    3.37284      None  ...             38.0   
3  54.907227  36.500176  -4.948787    4.40244      None  ...             38.0   
4  55.168495  36.500176  -4.948784    5.39892      None  ...             38.0   

  feature_dec_3mss  feature_acc_3_5mss  feature_de

## Nous avons regrouper les 2 fichiers csv et base de donnée.
Maintenant après exploration, nous allons supprimer, remplacer, recalculé certaine donnée abérante ou manquante.

### Vitesse abérante

In [ ]:
#Seuil de vitesse max à 50 km/h

con.execute(f"""
CREATE OR REPLACE VIEW sessions_aberrantes AS
SELECT
    player_name,
    session_date,
            sensor_id,
    MAX(speed_kmh) AS max_speed
FROM tracking_clean
GROUP BY player_name, session_date
HAVING MAX(speed_kmh) > 50;
""").df()
clean_vitesse = con.execute("""
SELECT *
FROM sessions_aberrantes
ORDER BY max_speed DESC
""").df()
print(clean_vitesse)

   player_name session_date  max_speed
0       SIDLER   2020-01-24  166.08996
1    CARBONELL   2020-11-18  149.78448
2        IBRAH   2020-09-16  147.07368
3         VOCA   2019-09-14  122.18940
4     CIRKOVIC   2019-07-11  108.66240
5        MALES   2019-09-18   85.11228
6     KNEZEVIC   2020-11-11   79.58520
7        ELEKE   2020-03-04   73.07172
8     KNEZEVIC   2021-01-08   69.80580
9       FRYDEK   2021-01-11   69.73056
10   SCHWEGLER   2021-01-08   69.29136
11      SORGIC   2021-01-08   69.05196
12   MARGIOTTA   2020-02-06   68.07708
13      SCHAUB   2021-01-08   66.98268
14       RYDER   2020-02-06   66.55104
15       ALVES   2021-01-08   66.24900
16     GRETHER   2021-01-08   63.87624
17     SCHURPF   2020-02-06   63.48456
18       ALVES   2020-11-04   59.59656
19   CARBONELL   2020-11-04   59.58072
20    KNEZEVIC   2020-11-04   59.08536
21    CIRKOVIC   2020-02-04   55.61964
22       ELEKE   2019-09-18   53.14140
23   KAKABADZE   2020-03-13   52.51320
24       RYDER   2020-03-

In [ ]:

df_speed_max = con.execute("""
SELECT
    player_name,
    MAX(speed_kmh) AS max_speed_kmh
FROM tracking_clean_speed
GROUP BY player_name
ORDER BY max_speed_kmh DESC
""").df()

print(df_speed_max)
print(df_speed_max.shape)

    player_name  max_speed_kmh
0      DEMHASAJ       43.18596
1     MARGIOTTA       42.06312
2      CIRKOVIC       40.00536
3         IBRAH       39.64932
4        NDENGE       39.55464
5          VOCA       39.47724
6      KNEZEVIC       39.41532
7        SCHULZ       39.31308
8         BURCH       36.87516
9     SCHNEUWLY       36.72900
10          TIA       35.60760
11        RYDER       35.41428
12      SCHURPF       35.21160
13        BÜRKI       34.93116
14    SCHWEGLER       34.44624
15      ALOUNGA       33.61104
16        TAZAR       33.51600
17       SIDLER       33.43500
18       BINOUS       33.00552
19    BALATUBAN       32.96412
20        ALVES       32.84712
21       SORGIC       32.37732
22            F       32.14728
23        MALES       31.98528
24    KAKABADZE       31.69008
25        ELEKE       31.60836
26         LINO       31.39308
27      GRETHER       31.04280
28         LANG       30.87540
29        ALABI       30.74904
30       FRYDEK       30.60504
31      

### Valeur manquante

In [1]:
#Recherchez toutes les valeurs Na
#Afficher une ligne en particulier
player_name = 'KNEZEVIC'
session_date = '2020-02-01'
na_values = con.execute(f"""
SELECT *
FROM tracking_clean
WHERE player_name = '{player_name}' AND session_date = '{session_date}'
""").df()
print(na_values)

na_summary = con.execute("""
SELECT
    COUNT(*) AS n_rows,
    SUM(player_name IS NULL)::INT AS na_player,
    SUM(session_date IS NULL)::INT AS na_session_date,
    SUM(time_utc IS NULL)::INT AS na_time,
    SUM(x_pos IS NULL)::INT AS na_x,
    SUM(y_pos IS NULL)::INT AS na_y,
    SUM(latitude IS NULL)::INT AS na_lat,
    SUM(longitude IS NULL)::INT AS na_lon,
    SUM(speed_kmh IS NULL)::INT AS na_speed
FROM tracking_clean
""").df()

print(na_summary)


NameError: name 'con' is not defined